In [1]:
cd /media/sharukathirimanne/OS/Users/Sharuka Thirimanne/Desktop/FYP-ML-IDS

/media/sharukathirimanne/OS/Users/Sharuka Thirimanne/Desktop/FYP-ML-IDS


In [2]:
import sklearn
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential
from tensorflow.keras import regularizers, Model, layers
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix,f1_score

#Pipelines
from sklearn.pipeline import make_pipeline

from pickle import dump, load
from sklearn.pipeline import Pipeline
from io import StringIO
from keras.models import model_from_json
import joblib


In [3]:
# Load the model
estimator = model_from_json(open('DNN_Realtime_Model.json','r').read())
estimator.load_weights('DNN_model_weights.h5')

# Load Column Transformer 
column_transformer = joblib.load('DNN_Realtime_ColumnTransformer')

In [4]:
METRICS = [ 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]
estimator.compile(loss='binary_crossentropy', optimizer='adam', metrics = METRICS)

In [5]:
pipe = Pipeline(steps=[
    ('preprocessing',column_transformer),
    ('classifier',estimator)])

In [6]:
from sklearn import set_config
set_config(display='diagram')
pipe

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['protocol_type', 'flag',
                                                   'service']),
                                                 ('standardscaler',
                                                  StandardScaler(),
                                                  ['duration', 'src_bytes',
                                                   'dst_bytes', 'land',
                                                   'wrong_fragment', 'urgent',
                                                   'count', 'srv_count',
                                                   'serror_rate',
                                                   'srv_serror_rate',
                                                   'rerror_rate',
                                                   'srv_rerror_rate',
                                                   'sam...
                                                   'diff_srv_rate',
                                                   'srv_diff_host_rate',
                                                   'dst_host_count',
                                                   'dst_host_srv_count',
                                                   'dst_host_same_srv_rate',
                                                   'dst_host_diff_srv_rate',
                                                   'dst_host_same_src_port_rate',
                                                   'dst_host_srv_diff_host_rate',
                                                   'dst_host_serror_rate',
                                                   'dst_host_srv_serror_rate',
                                                   'dst_host_rerror_rate',
                                                   'dst_host_srv_rerror_rate'])])),
                ('classifier',
                 <keras.engine.sequential.Sequential object at 0x7f359834e2e0>)])

In [7]:
col_list = ['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent',
            'count','srv_count','serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate','same_srv_rate',
            'diff_srv_rate','srv_diff_host_rate','dst_host_count','dst_host_srv_count','dst_host_same_srv_rate',
            'dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate',
            'dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate']

### Realtime IDS

In [8]:
ccc = ['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent','count','srv_count',
       'serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate','srv_diff_host_rate',
       'dst_host_count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate',
       'dst_host_srv_rerror_rate']

xx2 = "3,udp,other,SF,864,0,0,0,0,3,6,0.00,0.00,0.00,0.00,1.00,0.00,0.50,11,83,1.00,0.00,0.00,0.87,0.00,0.00,0.00,0.00"
str_data_n = StringIO(xx2)

df_tt = pd.read_csv(str_data_n,sep=",",names=ccc)
df_tt = df_tt[col_list]
df_tt

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,count,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,3,udp,other,SF,864,0,0,0,0,3,...,11,83,1.0,0.0,0.0,0.87,0.0,0.0,0.0,0.0


In [9]:
len(col_list)

28

In [10]:
test_predictions = pipe.predict(df_tt)
print('Predicted value : ',test_predictions)

if test_predictions>0.5:
    output = 'Anomaly'
else:
    output = 'Normal'
        
print('Status : ',output)

Predicted value :  [[5.9953194e-08]]
Status :  Normal
